# Testing the OTM API SDK

In [ ]:
from satvu_api_sdk.sdk import SatVuSDK
import os

sdk = SatVuSDK(
    client_id=os.getenv("SV_CLIENT_ID"),
    client_secret=os.getenv("SV_CLIENT_SECRET"),
    env="qa",
)
contract_id = ""
order_id = ""

## Full Order Flow

### Assured

#### Request Feasibility Check

In [ ]:
from satvu_api_sdk.services.otm.models import AssuredFeasibilityFields, FeasibilityRequest, Point
from datetime import datetime, timedelta

tomorrow = datetime.today() + timedelta(days=1)
six_days = datetime.today() + timedelta(days=6)

assured_feasibility_request = sdk.otm.post_tasking_feasibility(
    contract_id,
    FeasibilityRequest(
        geometry=Point(coordinates=[-3.5, 40.4]),
        properties=AssuredFeasibilityFields(datetime=f"{tomorrow}/{six_days}"),
    ),
)

#### Get Feasibility Response

In [ ]:
assured_feasibility_response = sdk.otm.get_tasking_feasibility_response(
    contract_id, assured_feasibility_request.id
)

#### Submit Order

In [ ]:
from satvu_api_sdk.services.otm.models import AssuredOrderRequest, AssuredOrderRequestProperties

assured_order = sdk.otm.post_tasking_orders(
    contract_id,
    AssuredOrderRequest(
        properties=AssuredOrderRequestProperties(
            signature=assured_feasibility_response.features[0].signature,
            name="My Assured Order",
        )
    ),
)

#### Edit Order

In [ ]:
from satvu_api_sdk.services.otm.models import EditOrderPayload, OrderName

sdk.otm.edit_tasking_order(
    contract_id,
    assured_order.id,
    EditOrderPayload(properties=OrderName(name="ABCD Order")),
)

#### Cancel Order

In [ ]:
sdk.otm.cancel_tasking_order(contract_id, assured_order.id)

### Standard

#### Request Feasibility Check

In [ ]:
from satvu_api_sdk.services.otm.models import (
    StandardRequestProperties,
    FeasibilityRequest,
    Point,
    DayNightMode,
)
from datetime import datetime, timedelta

tomorrow = datetime.today() + timedelta(days=1)
three_weeks = datetime.today() + timedelta(weeks=3)

standard_feasibility_request = sdk.otm.post_tasking_feasibility(
    contract_id,
    FeasibilityRequest(
        geometry=Point(coordinates=[-3.5, 40.4]),
        properties=StandardRequestProperties(
            datetime=f"{tomorrow}/{three_weeks}",
            max_cloud_cover=10,
            max_off_nadir=15,
            satvu_day_night_mode=DayNightMode.DAY,
        ),
    ),
)

#### Get Feasibility Response

In [ ]:
sdk.otm.get_tasking_feasibility_response(contract_id, standard_feasibility_request.id)

#### Submit Order

In [ ]:
from satvu_api_sdk.services.otm.models import StandardOrderRequest, StandardOrderRequestProperties

standard_order = sdk.otm.post_tasking_orders(
    contract_id,
    StandardOrderRequest(
        geometry=Point(coordinates=[-3.5, 40.4]),
        properties=StandardOrderRequestProperties(
            datetime=f"{tomorrow}/{three_weeks}",
            max_cloud_cover=10,
            satvu_day_night_mode=DayNightMode.DAY,
            max_off_nadir=15,
        ),
    ),
)